time signature (Audio)

Valence (Audio)

Danceability: tempo, chroma_stft

Energy: rms, spectral_centroid

Instrumentalness: mfcc (Higher variance in MFCCs might indicate instrumental tracks)

Loudness: rms

Speechiness: zcr, mfcc (Patterns in MFCCs that indicate speech)

Valence, Mood, Emotion: These are subjective and would require complex modeling with labeled data to infer accurately from the extracted features. Machine learning models trained on datasets where songs 
are labeled with these attributes can use these features as input to predict mood

In [1]:
import requests
from dotenv import load_dotenv
import os
import json
import base64   
import pandas as pd
import librosa
import numpy as np
import lyricsgenius
import langdetect
import re
import string
import tempfile

In [2]:
lastfm_api_key = "97d5a64d5ba4a8bc580b752ceff3b87f"
lastfm_secret = "35175090bd61f6f16ac607bd26e5b1de"

In [3]:
def search_deezer_track(track_name):
    search_url = "https://api.deezer.com/search/track"
    params = {"q": track_name}
    response = requests.get(search_url, params=params)
    
    if response.status_code == 200:
        tracks = response.json().get('data', [])
        for track in tracks:
            print(f"Track ID: {track['id']}, Title: {track['title']}, Artist: {track['artist']['name']}")
    else:
        print(f"Failed to search tracks. Status code: {response.status_code}")


def get_deezer_track_info(track_id):
    """
    Fetch track information and MP3 preview file URL from Deezer API.

    Parameters:
    - track_id: The unique identifier for the track on Deezer.

    Returns:
    A dictionary with track information and the preview URL.
    """
    base_url = "https://api.deezer.com/track/"
    response = requests.get(f"{base_url}{track_id}")
    
    if response.status_code == 200:
        data = response.json()
        main_artist = data.get("artist", {}).get("name", "")
        contributors = [contributor['name'] for contributor in data.get("contributors", []) if contributor['name'] != main_artist]
        featured_artists = ", ".join(contributors) if contributors else None

        track_info = {
            "title": data.get("title"),
            "artist": main_artist,
            "featured_artists": featured_artists,
            "duration": data.get("duration"),
            "album": data.get("album", {}).get("title"),
            "preview_url": data.get("preview"),
            "link": data.get("link")
        }
        return track_info
    else:
        print(f"Failed to fetch data for track ID {track_id}. Status code: {response.status_code}")
        return {}


def get_song_details_from_url(audio_url, api_token):
    """
    Fetch song details using AudD API from an audio URL.

    Parameters:
    - audio_url: URL to the audio file for song recognition.
    - api_token: Your AudD API token.

    Returns:
    A dictionary with song details.
    """
    api_endpoint = "https://api.audd.io/"
    params = {
        "url": audio_url,
        "return": "apple_music,spotify",  # You can specify what additional data you want (e.g., metadata from Apple Music or Spotify)
        "api_token": api_token,
    }

    response = requests.post(api_endpoint, data=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch song details. Status code: {response.status_code}")
        return {}



In [75]:
# search_deezer_track("Fur Elise")
get_deezer_track_info("4538913")

{'title': 'Für Elise',
 'artist': 'Ludwig van Beethoven',
 'featured_artists': None,
 'duration': 173,
 'album': 'Classical Best Of',
 'preview_url': 'https://cdns-preview-a.dzcdn.net/stream/c-a9dee2af0ec7ab7000e8f63cca353ce7-13.mp3',
 'link': 'https://www.deezer.com/track/4538913'}

In [139]:
def extract_librosa_features_from_url(url):
    # Fetch the audio file from the URL
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to download audio file from {url}")
    
    # Create a temporary file and manually manage it
    temp_dir = tempfile.gettempdir()
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3", dir=temp_dir)
    temp_file_path = temp_file.name
    
    try:
        # Write the fetched content to the temp file and close it to release the lock
        temp_file.write(response.content)
        temp_file.close()
        
        # Now, load the audio file from the path
        y, sr = librosa.load(temp_file_path, sr=None)  # Using sr=None to preserve the original sampling rate
        
        # Analyze the audio file
        duration = librosa.get_duration(y=y, sr=sr)
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        harmonic, percussive = librosa.effects.hpss(y)
        rms = np.mean(librosa.feature.rms(y=y))
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y))
        beat_times = librosa.frames_to_time(beat_frames, sr=sr)
        
        tonnetz = np.mean(librosa.feature.tonnetz(y=y, sr=sr).T, axis=0)
        spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr).T, axis=0)
        
        harmonic_centroid = librosa.feature.spectral_centroid(y=harmonic, sr=sr)
        percussive_centroid = librosa.feature.spectral_centroid(y=percussive, sr=sr)


        mfcc_var = np.std(librosa.feature.mfcc(y=y, sr=sr).T, axis=0)
        chroma_var = np.std(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)

        features = {
            'mfcc': np.mean(librosa.feature.mfcc(y=y, sr=sr).T, axis=0),
            'mfcc_var': mfcc_var, 
            'chroma': np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0),
            'chroma_var': chroma_var, 
            'duration': duration,
            'rms': rms,
            'spectral_bandwidth': spectral_bandwidth,
            'spectral_contrast': spectral_contrast,
            'spectral_centroid': spectral_centroid,
            'harmonic_centroid': harmonic_centroid,
            'percussive_centroid': percussive_centroid,
            'zcr': zcr,
            'tonnetz': tonnetz,
            'tempo': tempo,
            'beat_times': beat_times
        }
    finally:
        # Ensure the temporary file is removed after processing
        os.remove(temp_file_path)

    return features

In [140]:
import numpy as np

def estimate_instrumentalness(features):
    harmonic_centroid = features['harmonic_centroid']
    percussive_centroid = features['percussive_centroid']

    harmonic_mean_centroid = np.mean(harmonic_centroid)
    percussive_mean_centroid = np.mean(percussive_centroid)

    instrumentalness = 1.0 - (percussive_mean_centroid / harmonic_mean_centroid)

    return instrumentalness


def estimate_speechiness(features):
    # Extract relevant features
    zcr_mean = np.mean(features['zcr'])
    spectral_centroid_mean = np.mean(features['spectral_centroid'])
    spectral_bandwidth_mean = np.mean(features['spectral_bandwidth'])

    # Calculate statistical properties of features
    mean_zcr = np.mean(zcr_mean)
    mean_centroid = np.mean(spectral_centroid_mean)
    mean_bandwidth = np.mean(spectral_bandwidth_mean)

    # Calculate speechiness score based on dynamic heuristic approach
    # This approach estimates speechiness based on deviations from the mean of features
    # The assumption is that speech will exhibit certain characteristics in terms of zero crossing rate, spectral centroid, and spectral bandwidth

    # Calculate deviations from mean
    zcr_deviation = (mean_zcr - zcr_mean) / (np.std(zcr_mean) + 1e-8)
    centroid_deviation = (mean_centroid - spectral_centroid_mean) / (np.std(spectral_centroid_mean) + 1e-8)
    bandwidth_deviation = (mean_bandwidth - spectral_bandwidth_mean) / (np.std(spectral_bandwidth_mean) + 1e-8)

    # Aggregate deviations and compute speechiness score
    deviation_score = np.mean([zcr_deviation, centroid_deviation, bandwidth_deviation])
    speechiness_score = 1.0 - np.clip(deviation_score, 0.0, 1.0)  # Clip to [0, 1]

    return speechiness_score






def soft_normalize(value, scale=1.0, offset=0.0):
    """Applies a soft normalization with optional scaling and offset."""
    # Ensure value is non-negative and apply a logarithmic scaling for normalization
    return np.log1p(max(0, value)) / (np.log1p(scale) + offset)

def calculate_song_features(features):
    scores = {}

    # Danceability
    beat_intervals = np.diff(features['beat_times'])
    beat_interval_variability = 1 / (1 + np.var(beat_intervals))  # Inverted variability for regularity
    scores['danceability'] = np.mean([
        1 - (features['tempo'] / 240),  # Invert as higher tempo doesn't always mean more danceable
        beat_interval_variability,
        np.mean(features['chroma'])  # Chroma contributes to danceability
    ])

    # Energy
    dynamic_range = np.max(features['rms']) - np.min(features['rms'])
    perceived_loudness = features['rms']
    timbre = features['spectral_centroid'] / np.max(features['spectral_centroid'])
    onset_rate = len(features['beat_times']) / features['duration']
    scores['energy'] = np.mean([
        dynamic_range,
        perceived_loudness,
        timbre,
        onset_rate
    ])

    # Instrumentalness

    instrumentalness_score = estimate_instrumentalness(features)
    scores['instrumentalness'] = instrumentalness_score



    # Loudness (Using RMS as a proxy, normalized to a -60 to 0 dB range for demonstration)
    # This is an approximation and may not accurately reflect Spotify's loudness in dB.
    rms_db = 20 * np.log10(features['rms'] + 1e-6)  # Avoid log(0) by adding a small constant
    scores['loudness'] = np.interp(rms_db, [-60, 0], [0, 1])  # Interpolate RMS dB value to 0-1 range

    # Speechiness
    # Extract features
    speechiness_score = estimate_speechiness(features)
    scores['speechiness'] = speechiness_score


    return scores



In [141]:
def analyze_artist_tracks(artist_name, song_name):
    # def search_deezer_track(track_name):
    #     search_url = "https://api.deezer.com/search/track"
    #     params = {"q": track_name}
    #     response = requests.get(search_url, params=params)
    #     tracks = []
    #     if response.status_code == 200:
    #         tracks_data = response.json().get('data', [])
    #         for track in tracks_data:
    #             if artist_name.lower() in track['artist']['name'].lower():
    #                 tracks.append({
    #                     'id': track['id'],
    #                     'title': track['title'],
    #                     'artist': track['artist']['name'],
    #                     'preview_url': track['preview']
    #                 })
    #     return tracks

    def search_deezer_track(track_name):
        search_url = "https://api.deezer.com/search/track"
        query = f'artist:"{artist_name}" track:"{song_name}"'
        params = {"q": query}
        response = requests.get(search_url, params=params)
        if response.status_code == 200:
            tracks_data = response.json().get('data', [])
            for track in tracks_data:
                if artist_name.lower() in track['artist']['name'].lower():
                    # Return the first match immediately
                    return {
                        'id': track['id'],
                        'title': track['title'],
                        'artist': track['artist']['name'],
                        'preview_url': track['preview']
                    }
        return None  # Return None if no match is found

    # Search for the track
    track_name = f"{artist_name} {song_name}"
    track = search_deezer_track(track_name)

    # Initialize DataFrame
    df = pd.DataFrame(columns=['Song Name', 'Artist Name', 'Danceability', 'Energy', 'Instrumentalness', 'Loudness', 'Speechiness'])

    # Process each track found
    if track:
        # Extract features
        features = extract_librosa_features_from_url(track['preview_url'])

        # Calculate scores
        score = calculate_song_features(features)

        # Append to DataFrame directly without for-loop
        df = df.append({
            'Song Name': track['title'],
            'Artist Name': track['artist'],
            'Danceability': score.get('danceability', np.nan),
            'Energy': score.get('energy', np.nan),
            'Instrumentalness': score.get('instrumentalness', np.nan),
            'Loudness': score.get('loudness', np.nan),
            'Speechiness': score.get('speechiness', np.nan)
        }, ignore_index=True)

    return df


In [144]:
# analyze_artist_tracks("Ludwig van Beethoven", "Fur Elise")
# analyze_artist_tracks("IU", "Blueming")
analyze_artist_tracks("Psy", "Gangnam Style")

C:\Users\wyero\AppData\Local\Temp\ipykernel_8504\2371916571.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({


,Song Name,Artist Name,Danceability,Energy,Instrumentalness,Loudness,Speechiness
0,Gangnam style,Psy,0.64482,0.880602,-0.326651,0.829844,1.0
